In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import mlflow
import mlflow.pytorch

# Set the MLflow tracking URI (use your own server or the default local URI)
mlflow.set_tracking_uri("http://host.docker.internal:5000")

# Create a MLflow experiment (you can change the experiment name)
mlflow.set_experiment("simple-network")

# Define hyperparameters
learning_rate = 0.001
num_epochs = 1
batch_size = 64

# Create a PyTorch neural network model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(784, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = SimpleNet()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Load and preprocess the dataset (MNIST as an example)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
with mlflow.start_run() as run:
    mlflow.log_params({"learning_rate": learning_rate, "num_epochs": num_epochs, "batch_size": batch_size})

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Log metrics and save the model
        mlflow.log_metric("train_loss", loss.item(), step=epoch)
        mlflow.pytorch.log_model(model, "models")

print("experiment run finished")

# You can view the results in the MLflow UI by running `mlflow ui` in your terminal


experiment run finished
